In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib widget
basePath ='./sims.TNG/TNG50-1/output'
from tqdm import tqdm
import gc

fields=['SubhaloPos','SubhaloMass']
subfindid=448830
axis='xy'
import os

if not os.path.exists('figures/' + str(subfindid)):
    os.makedirs('figures/' +str(subfindid))

In [2]:
class Cell:
    def __init__(self):
        self.Coordinate = [0, 0]
        self.Mass = 0
        self.Temperature = 0

class Cloudyz:
    def __init__(self, x_range, y_range):
        x_min, x_max = x_range
        y_min, y_max = y_range
        x_centers = np.arange(x_min + 0.25, x_max, 0.5)
        y_centers = np.arange(y_min + 0.25, y_max, 0.5)
        xx, yy = np.meshgrid(x_centers, y_centers)
        self.cells = [Cell() for _ in range(xx.size)]
        self.x_range = x_range
        self.y_range = y_range
        self.assign_coordinates(xx.flatten(), yy.flatten())

    def assign_coordinates(self, x_coords, y_coords):
        for i, cell in enumerate(self.cells):
            cell.Coordinate = np.array([y_coords[i], x_coords[i]])

    def assign_parameter(self, coordinates: np.ndarray,masses: np.ndarray, temperature: np.ndarray):
        x_min, x_max = self.x_range
        y_min, y_max = self.y_range
        x_bars = int((x_max - x_min) / 0.5)
        for i in range(coordinates.shape[0]):
            coord = coordinates[i]
            param = temperature[i]
            mass=masses[i]
            x_index = int((coord[1] - x_min) / 0.5)
            y_index = int((coord[2] - y_min) / 0.5)
            if x_index >= x_bars or y_index >= x_bars or x_index<0 or y_index < 0:
                continue
            if param > 10000:
                continue
            index = x_index * x_bars + y_index
            self.cells[index].Temperature = param
            self.cells[index].Mass += mass

            
    def assign_mass(self, coordinates: np.ndarray,masses: np.ndarray):
        x_min, x_max = self.x_range
        y_min, y_max = self.y_range
        x_bars = int((x_max - x_min) / 0.5)
        for i in range(coordinates.shape[0]):
            coord = coordinates[i]
            mass=masses[i]
            x_index = int((coord[1] - x_min) / 0.5)
            y_index = int((coord[2] - y_min) / 0.5)
            if x_index >= x_bars or y_index >= x_bars or x_index<0 or y_index < 0:
                continue
            index = x_index * x_bars + y_index
            self.cells[index].Mass += mass    
    
    
    
    def visualize(self, savePath):
        plt.figure(figsize=(20, 20), dpi=300)
        plt.style.use("dark_background")
        x_range, y_range = self.x_range, self.y_range
        plt.hist2d([cell.Coordinate[0] for cell in self.cells], [cell.Coordinate[1] for cell in self.cells], norm=mpl.colors.LogNorm(),cmin=0.00005, bins=[int((x_range[1]-x_range[0])*2),int((y_range[1]-y_range[0])*2)], weights=[cell.Mass for cell in self.cells]) 
        plt.xlabel('y [ckpc/h]')
        plt.ylabel('z [ckpc/h]')
        plt.ylim(self.x_range[0],self.x_range[1])
        plt.xlim(self.y_range[0],self.y_range[1])
        plt.axes().get_xaxis().set_visible(False)
        plt.axes().get_yaxis().set_visible(False)
        plt.axis('equal')
        plt.savefig(savePath)
        plt.close('all')

    def reset(self):
        for cell in self.cells:
            cell.Coordinate = [0, 0]
            cell.Mass = 0
            cell.Temperature = 0
       
class Cloudxy:
    def __init__(self, x_range, y_range):
        x_min, x_max = x_range
        y_min, y_max = y_range
        x_centers = np.arange(x_min + 0.25, x_max, 0.5)
        y_centers = np.arange(y_min + 0.25, y_max, 0.5)
        xx, yy = np.meshgrid(x_centers, y_centers)
        self.cells = [Cell() for _ in range(xx.size)]
        self.x_range = x_range
        self.y_range = y_range
        self.assign_coordinates(xx.flatten(), yy.flatten())

    def assign_coordinates(self, x_coords, y_coords):
        for i, cell in enumerate(self.cells):
            cell.Coordinate = np.array([y_coords[i], x_coords[i]])

    def assign_parameter(self, coordinates: np.ndarray,masses: np.ndarray, temperature: np.ndarray):
        x_min, x_max = self.x_range
        y_min, y_max = self.y_range
        x_bars = int((x_max - x_min) / 0.5)
        for i in range(coordinates.shape[0]):
            coord = coordinates[i]
            param = temperature[i]
            mass=masses[i]
            x_index = int((coord[0] - x_min) / 0.5)
            y_index = int((coord[1] - y_min) / 0.5)
            if x_index >= x_bars or y_index >= x_bars or x_index<0 or y_index < 0:
                continue
            if param > 10000:
                continue
            index = x_index * x_bars + y_index
            self.cells[index].Temperature = param
            self.cells[index].Mass += mass

            
    def assign_mass(self, coordinates: np.ndarray,masses: np.ndarray):
        x_min, x_max = self.x_range
        y_min, y_max = self.y_range
        x_bars = int((x_max - x_min) / 0.5)
        for i in range(coordinates.shape[0]):
            coord = coordinates[i]
            mass=masses[i]
            x_index = int((coord[0] - x_min) / 0.5)
            y_index = int((coord[1] - y_min) / 0.5)
            if x_index >= x_bars or y_index >= x_bars or x_index<0 or y_index < 0:
                continue
            index = x_index * x_bars + y_index
            self.cells[index].Mass += mass    
    
    
    
    def visualize(self, savePath):
        plt.figure(figsize=(20, 20), dpi=300)
        plt.style.use("dark_background")
        x_range, y_range = self.x_range, self.y_range
        plt.hist2d([cell.Coordinate[0] for cell in self.cells], [cell.Coordinate[1] for cell in self.cells], norm=mpl.colors.LogNorm(),cmin=0.00005, bins=[int((x_range[1]-x_range[0])*2),int((y_range[1]-y_range[0])*2)], weights=[cell.Mass for cell in self.cells]) 
        plt.xlabel('y [ckpc/h]')
        plt.ylabel('z [ckpc/h]')
        plt.ylim(self.x_range[0],self.x_range[1])
        plt.xlim(self.y_range[0],self.y_range[1])
        plt.axes().get_xaxis().set_visible(False)
        plt.axes().get_yaxis().set_visible(False)
        plt.axis('equal')
        plt.savefig(savePath)
        plt.close('all')

    def reset(self):
        for cell in self.cells:
            cell.Coordinate = [0, 0]
            cell.Mass = 0
            cell.Temperature = 0
    
    
    
class Cloudxz:
    def __init__(self, x_range, y_range):
        x_min, x_max = x_range
        y_min, y_max = y_range
        x_centers = np.arange(x_min + 0.25, x_max, 0.5)
        y_centers = np.arange(y_min + 0.25, y_max, 0.5)
        xx, yy = np.meshgrid(x_centers, y_centers)
        self.cells = [Cell() for _ in range(xx.size)]
        self.x_range = x_range
        self.y_range = y_range
        self.assign_coordinates(xx.flatten(), yy.flatten())

    def assign_coordinates(self, x_coords, y_coords):
        for i, cell in enumerate(self.cells):
            cell.Coordinate = np.array([y_coords[i], x_coords[i]])

    def assign_parameter(self, coordinates: np.ndarray,masses: np.ndarray, temperature: np.ndarray):
        x_min, x_max = self.x_range
        y_min, y_max = self.y_range
        x_bars = int((x_max - x_min) / 0.5)
        for i in range(coordinates.shape[0]):
            coord = coordinates[i]
            param = temperature[i]
            mass=masses[i]
            x_index = int((coord[0] - x_min) / 0.5)
            y_index = int((coord[2] - y_min) / 0.5)
            if x_index >= x_bars or y_index >= x_bars or x_index<0 or y_index < 0:
                continue
            if param > 10000:
                continue
            index = x_index * x_bars + y_index
            self.cells[index].Temperature = param
            self.cells[index].Mass += mass

            
    def assign_mass(self, coordinates: np.ndarray,masses: np.ndarray):
        x_min, x_max = self.x_range
        y_min, y_max = self.y_range
        x_bars = int((x_max - x_min) / 0.5)
        for i in range(coordinates.shape[0]):
            coord = coordinates[i]
            mass=masses[i]
            x_index = int((coord[0] - x_min) / 0.5)
            y_index = int((coord[2] - y_min) / 0.5)
            if x_index >= x_bars or y_index >= x_bars or x_index<0 or y_index < 0:
                continue
            index = x_index * x_bars + y_index
            self.cells[index].Mass += mass    
    
    
    
    def visualize(self, savePath):
        plt.figure(figsize=(20, 20), dpi=300)
        plt.style.use("dark_background")
        x_range, y_range = self.x_range, self.y_range
        plt.hist2d([cell.Coordinate[0] for cell in self.cells], [cell.Coordinate[1] for cell in self.cells], norm=mpl.colors.LogNorm(),cmin=0.00005, bins=[int((x_range[1]-x_range[0])*2),int((y_range[1]-y_range[0])*2)], weights=[cell.Mass for cell in self.cells]) 
        plt.xlabel('y [ckpc/h]')
        plt.ylabel('z [ckpc/h]')
        plt.ylim(self.x_range[0],self.x_range[1])
        plt.xlim(self.y_range[0],self.y_range[1])
        plt.axes().get_xaxis().set_visible(False)
        plt.axes().get_yaxis().set_visible(False)
        plt.axis('equal')
        plt.savefig(savePath)
        plt.close('all')

    def reset(self):
        for cell in self.cells:
            cell.Coordinate = [0, 0]
            cell.Mass = 0
            cell.Temperature = 0
    
def Binary_DataIter(Data_Set1,Data_Set2):
    for i in range(0,min(len(Data_Set1),len(Data_Set2))):
        yield Data_Set1[i],Data_Set2[i]
               

In [3]:
tree_fields=['SubfindID','SnapNum']
subfindid_snapnum_tree=il.sublink.loadTree(basePath, 99, subfindid, fields=tree_fields, onlyMPB=True)
subfind_id=subfindid_snapnum_tree['SubfindID']
snapshot_num=subfindid_snapnum_tree['SnapNum']
del subfindid_snapnum_tree
gc.collect()

0

In [4]:
for current_subfind_id,current_snapshot_num in Binary_DataIter(subfind_id,snapshot_num):
    if current_snapshot_num < 67: break
    if current_snapshot_num >=73:continue
    
    
    
    subhalo_data=il.groupcat.loadSubhalos(basePath, current_snapshot_num, fields=fields)
    Positions=subhalo_data['SubhaloPos']
    Mass=subhalo_data['SubhaloMass']
    GalaxyPos=Positions[current_subfind_id]
    
    if axis=='xy':
        xmin=GalaxyPos[0]-550
        xmax=GalaxyPos[0]+550
    
        ymin=GalaxyPos[1]-550
        ymax=GalaxyPos[1]+550
    
        condition = (Positions[:, 0] >= xmin) & (Positions[:, 0] <= xmax) & (Positions[:, 1] >= ymin) & (Positions[:, 1] <= ymax) & (Mass > 0.01)
        index = np.where(condition)[0]

        del subhalo_data,Positions,Mass,condition,GalaxyPos
        gc.collect()
        cloud=Cloudxy([int(xmin+50),int(xmin+1050)],[int(ymin+50),int(ymin+1050)])
    
    
    if axis=='xz':
        xmin=GalaxyPos[0]-550
        xmax=GalaxyPos[0]+550
    
        ymin=GalaxyPos[2]-550
        ymax=GalaxyPos[2]+550
    
        condition = (Positions[:, 0] >= xmin) & (Positions[:, 0] <= xmax) & (Positions[:, 2] >= ymin) & (Positions[:, 2] <= ymax) & (Mass > 0.01)
        index = np.where(condition)[0]

        del subhalo_data,Positions,Mass,condition,GalaxyPos
        gc.collect()
        cloud=Cloudxz([int(xmin+50),int(xmin+1050)],[int(ymin+50),int(ymin+1050)])
    
    if axis=='yz':
        xmin=GalaxyPos[1]-550
        xmax=GalaxyPos[1]+550
    
        ymin=GalaxyPos[2]-550
        ymax=GalaxyPos[2]+550
    
        condition = (Positions[:, 1] >= xmin) & (Positions[:, 1] <= xmax) & (Positions[:, 2] >= ymin) & (Positions[:, 2] <= ymax) & (Mass > 0.01)
        index = np.where(condition)[0]

        del subhalo_data,Positions,Mass,condition,GalaxyPos
        gc.collect()
        cloud=Cloudyz([int(xmin+50),int(xmin+1050)],[int(ymin+50),int(ymin+1050)])
        
    for i in tqdm(index):
        cell_data=il.snapshot.loadSubhalo(basePath, current_snapshot_num, i, 'gas', fields=['Coordinates','Masses'])#,'ElectronAbundance','InternalEnergy','Masses'])
        if len(cell_data)==1 :continue
        
        GasCoordinates = cell_data['Coordinates'].astype(np.float64)
        Masses = cell_data['Masses'].astype(np.float64)
#        x_e=cell_data['ElectronAbundance'].astype(np.float64)
#        internal_energy=cell_data['InternalEnergy'].astype(np.float64)
#        gas_cell_masses=cell_data['Masses'].astype(np.float64)
    
#        m_p=1.673E-24
#        X_H=0.76
#        unit_switching=1E10
#        mean_molecular_weight=4*m_p/(1+3*X_H+4*X_H*x_e)
#        k_B=1.38E-16
#        gas_cell_temperature_in_Kelvin=2/3*internal_energy/k_B*unit_switching*mean_molecular_weight

        cloud.assign_mass(GasCoordinates, Masses)
        del GasCoordinates,Masses,cell_data#,x_e,internal_energy,gas_cell_masses,mean_molecular_weight,gas_cell_temperature_in_Kelvin

        
    del index
    if not os.path.exists('figures/' + str(subfindid)+'/'+axis):
        os.makedirs('figures/' +str(subfindid)+'/'+axis)
    cloud.visualize('./figures/'+str(subfindid)+'/'+axis+'/{}.png'.format(current_snapshot_num))
    del cloud
    gc.collect()

100%|██████████| 1099/1099 [00:27<00:00, 39.85it/s]
/opt/conda/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/opt/conda/lib/python3.6/site-packages/matplotlib/colors.py:1031: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0
100%|██████████| 1065/1065 [00:29<00:00, 36.41it/s]
